In [1]:
from pgsql_search.loader import HuggingFaceDatasets

ds = HuggingFaceDatasets("UCSC-VLAA/Recap-COCO-30K")
ds.dataset = ds.dataset.shuffle()
ds = ds.select(list(range(100)))
ds.save_images("../data/images100")
ds = ds.select_columns(["image_filepath", "caption"])
ds.dataset


2024-12-18 17:07:51.948 | INFO     | pgsql_search.loader:__init__:15 - Loading dataset: UCSC-VLAA/Recap-COCO-30K
2024-12-18 17:07:51.956 | INFO     | pgsql_search.loader:save_images:44 - Saving images to folder: ../data/images100


Saving images (num_proc=8):   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset({
    features: ['image_filepath', 'caption'],
    num_rows: 100
})

In [2]:
df = ds.dataset.to_pandas()
df

,image_filepath,caption
0,../data/images100/364010.jpg,Two small dogs sitting on a towel in the back ...
1,../data/images100/525909.jpg,The orange cat looks at a television set in a ...
2,../data/images100/40881.jpg,A bowl is filled with meat and different types...
3,../data/images100/252986.jpg,Small wooden bench sitting in the middle of a ...
4,../data/images100/578418.jpg,A delicious looking salad with tuna and bread.
...,...,...
95,../data/images100/39502.jpg,An outdoor kitchen with a furnace and ice box.
96,../data/images100/436164.jpg,a horse pulls a cart with some people sitting ...
97,../data/images100/379077.jpg,A dog standing in the snow with some frisbees.
98,../data/images100/464251.jpg,a nice bird standing on a bench gazing at


In [3]:
from pgsql_search.database import PostgreSQLDatabase, ColumnType

In [4]:
with PostgreSQLDatabase("retrieval_db") as db:
    # First, create the table with just an ID column
    db.initialize_table("image_metadata")
    db.add_column("image_filepath", ColumnType.TEXT, nullable=False)
    db.add_column("caption", ColumnType.TEXT, nullable=True)


    db.insert_dataframe(df)

2024-12-18 17:07:52.478 | INFO     | pgsql_search.database:connect:87 - Connected to database
2024-12-18 17:07:52.482 | INFO     | pgsql_search.database:setup_pgvector_extension:104 - pgvector extension initialized
2024-12-18 17:07:52.486 | INFO     | pgsql_search.database:initialize_table:126 - Initialized table 'image_metadata' with ID column 'id'
2024-12-18 17:07:52.488 | INFO     | pgsql_search.database:add_columns:175 - Added 1 new columns to image_metadata
2024-12-18 17:07:52.489 | INFO     | pgsql_search.database:add_columns:175 - Added 1 new columns to image_metadata
2024-12-18 17:07:52.501 | INFO     | pgsql_search.database:insert_dataframe:227 - Inserted batch 1 (100/100 rows)
2024-12-18 17:07:52.501 | INFO     | pgsql_search.database:insert_dataframe:231 - Successfully inserted 100 rows into image_metadata
2024-12-18 17:07:52.501 | INFO     | pgsql_search.database:disconnect:96 - Disconnected from database
